In [1]:
import random
import mysql.connector
import json
import string
import datetime as dt
from pymongo import MongoClient
from pprint import pprint

def getGenero(argument):
    switcher = {        
        5: "action",
        6: "adventure",
        7: "casual",
        8: "strategy",
        9: "RPG",
        10: "simulation",
        11: "early_access",
        12: "free_to_play",
        13: "sports",
        14: "racing",
        15: "massively_multiplayer"
    }
    return switcher.get(argument, argument)

myclient = MongoClient("mongodb://localhost:27017/")
mydbmongo = myclient['stum_for_you']
mycol = mydbmongo["usuarios"]
colmul = mydbmongo["multidimensional"]

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    database='stum_for_you',
    passwd=""
)
from datetime import datetime,timedelta
hoy = datetime.now()
dias = timedelta(days=7)
semana_pasada = hoy-dias  
#se comprueba que sea domingo (0)
comp=hoy.strftime("%w")
print(comp)
if (comp == '0') :#--------------------------------------------------------------cuidado
    
    hoy=hoy.strftime("%Y-%m-%d")
    #semana_pasada=semana_pasada.strftime("%Y:%m:%d")
    
    semana_pasada ="2017-01-01"#-------------------------------------------------para pruebas
    print(hoy,semana_pasada)
    mycursor = mydb.cursor(buffered=True)

    mycursor.execute("SELECT * FROM transacciones_venta WHERE fecha_venta BETWEEN %s AND  %s", (semana_pasada,hoy,))
    transacciones = mycursor.fetchall()
    tipo_movimiento = "venta"
    
    for t in transacciones :
        es_juego = False
        es_dlc = False
        existe_descuento = False
        existe_descuento_personalizado = False
   
    #"movimiento"
        precio_final = t[1]
        print(t[0])
        mycursor.execute("SELECT * FROM venta_juegos WHERE id_transaccion = %s", (t[0],))
        venta = mycursor.fetchall()
        cantidad = len(venta)
    #fecha
        anyo=t[2].strftime("%Y")
        mes=t[2].strftime("%m")
        dia=t[2].strftime("%d")
    #juego
        if(len(venta)>0) :
            es_juego = True
            mycursor.execute("SELECT id_juego FROM claves_juegos WHERE id_clave = %s", (venta[0][1],))
            puntero_juego = mycursor.fetchone()[0]
            mycursor.execute("SELECT * FROM juegos WHERE id_juego = %s", (puntero_juego,))
            juego = mycursor.fetchone()
            genero= []
            #sacar genero
            for a in range (5,15) :
                if (juego[a]==True) :
                    genero.append(getGenero(a))
            precio_juego=juego[3]
            clasificacion=juego[1]
            #descuento_personalizado
            puntero_usuario = venta[0][2]
            descuento_personalizado = mycol.find_one({"id_usuario": puntero_usuario, "$or" :[{"cupon":{"tipo": "all"}},{"cupon":{"tipo": "dlc"}}]})
            if descuento_personalizado:
                existe_descuento_personalizado = True
                porcentaje_personalizado = descuento_personalizado["porcentaje"]
                fecha_inicio_personalizado = descuento_personalizado["fecha_inicio"]
                fecha_fin_personalizado = descuento_personalizado["fecha_fin"]
                tipo_personalizado = descuento_personalizado["tipo"]
            
            #descuento
            mycursor.execute("SELECT d.* FROM descuentos d JOIN claves_juegos c ON c.id_juego = d.id_juego WHERE c.id_clave = %s", (venta[0][1],))
            descuento = mycursor.fetchone()
            if descuento:
                existe_descuento = True
                porcentaje = descuento[1]
                fecha_inicio = descuento[2].strftime("%Y-%m-%d")
                fecha_fin = descuento[3].strftime("%Y-%m-%d")

     #dlc
        if(len(venta)<=0) :
            es_dlc = True
            mycursor.execute("SELECT * FROM venta_dlcs WHERE id_tansaccion = %s", (t[0],))
            venta = mycursor.fetchall()
            cantidad = len(venta)
            mycursor.execute("SELECT id_dlc FROM claves_dlc WHERE id_clave = %s", (venta[0][0],))
            puntero_dlc = mycursor.fetchone()[0]
            mycursor.execute("SELECT * FROM dlcs WHERE id_dlc = %s", (puntero_dlc,))
            dlc = mycursor.fetchone()
            tipo= dlc[3]
            precio_dlc=dlc[1]
            #descuento_personalizado
            puntero_usuario = venta[0][2]
            descuento_personalizado = mycol.find_one({"id_usuario": puntero_usuario, "$or" :[{"cupon":{"tipo": "all"}},{"cupon":{"tipo": "dlc"}}]})
            if descuento_personalizado:
                existe_descuento_personalizado = True
                porcentaje_personalizado = descuento_personalizado["porcentaje"]
                fecha_inicio_personalizado = descuento_personalizado["fecha_inicio"]
                fecha_fin_personalizado = descuento_personalizado["fecha_fin"]
                tipo_personalizado = descuento_personalizado["tipo"]
            
            #descuento
            mycursor.execute("SELECT d.* FROM descuentos d JOIN claves_dlc c ON c.id_dlc = d.id_dlc WHERE c.id_clave = %s", (venta[0][0],))
            descuento = mycursor.fetchone()
            if descuento:
                existe_descuento = True
                porcentaje = descuento[1]
                fecha_inicio = descuento[2].strftime("%Y-%m-%d")
                fecha_fin = descuento[3].strftime("%Y-%m-%d")
            
    #usuario
        puntero_usuario=venta[0][2]
        usuario = mycol.find_one({"id_usuario": puntero_usuario})
        sexo = usuario["sexo"]
        fechaNacimiento = usuario["fecha_nacimiento"]
        #ubicacion
        pais = usuario["pais"]
        ciudad = usuario["ciudad"]

        datos = {
            "movimiento":{"precio_final":precio_final, "tipo_movimiento": tipo_movimiento, "cantidad": cantidad},
            "fecha":{"anyo":anyo, "mes":mes,"dia":dia},
            "ubicacion":{"pais":pais,"ciudad":ciudad},
            "usuario":{"sexo":sexo,"fechaNacimiento":fechaNacimiento,"pais":pais,"ciudad":ciudad},
            }
        if es_juego:
            datos["juego"] = {"puntero_juego":puntero_juego,"genero":genero,"precio":precio_juego,"clasificacion":clasificacion}
        if es_dlc:
            datos["dlc"] = {"precio":precio_dlc,"tipo":tipo}
        if existe_descuento:
            datos["descuento"] = {"porcentaje":porcentaje,"fecha_inicio":fecha_inicio,"fecha_fin":fecha_fin}
        if existe_descuento_personalizado:
            datos["descuento_personalizado"] = {"porcentaje":porcentaje_personalizado,"fecha_inicio":fecha_inicio_personalizado,"fecha_fin":fecha_fin_personalizado, "tipo":tipo_personalizado}
        
        print("---------------------------------------------------------------")
        pprint(datos)
        x = colmul.insert_one(datos) 
    

0
2020-01-12 2017-01-01
1
---------------------------------------------------------------
{'fecha': {'anyo': '2019', 'dia': '10', 'mes': '12'},
 'juego': {'clasificacion': 'INDIE',
           'genero': ['adventure'],
           'precio': 2,
           'puntero_juego': 9179},
 'movimiento': {'cantidad': 1, 'precio_final': 0, 'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Alacant', 'pais': 'Spain'},
 'usuario': {'ciudad': 'Alacant',
             'fechaNacimiento': '1999-09-13',
             'pais': 'Spain',
             'sexo': 'H'}}
2
---------------------------------------------------------------
{'fecha': {'anyo': '2019', 'dia': '18', 'mes': '07'},
 'juego': {'clasificacion': 'INDIE',
           'genero': ['action'],
           'precio': 5,
           'puntero_juego': 8611},
 'movimiento': {'cantidad': 1, 'precio_final': 0, 'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Ceuta', 'pais': 'Spain'},
 'usuario': {'ciudad': 'Ceuta',
             'fechaNacimiento': '1988-07-1

---------------------------------------------------------------
{'fecha': {'anyo': '2019', 'dia': '24', 'mes': '05'},
 'juego': {'clasificacion': 'INDIE',
           'genero': ['action', 'early_access'],
           'precio': 10,
           'puntero_juego': 4961},
 'movimiento': {'cantidad': 1, 'precio_final': 0, 'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Murcia', 'pais': 'Spain'},
 'usuario': {'ciudad': 'Murcia',
             'fechaNacimiento': '1990-10-17',
             'pais': 'Spain',
             'sexo': 'H'}}
27
---------------------------------------------------------------
{'fecha': {'anyo': '2019', 'dia': '01', 'mes': '02'},
 'juego': {'clasificacion': 'AAA',
           'genero': ['casual'],
           'precio': 10,
           'puntero_juego': 5550},
 'movimiento': {'cantidad': 1, 'precio_final': 0, 'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Murcia', 'pais': 'Spain'},
 'usuario': {'ciudad': 'Murcia',
             'fechaNacimiento': '1992-03-14',
        

---------------------------------------------------------------
{'fecha': {'anyo': '2019', 'dia': '19', 'mes': '11'},
 'juego': {'clasificacion': 'INDIE',
           'genero': ['adventure'],
           'precio': 10,
           'puntero_juego': 1635},
 'movimiento': {'cantidad': 1, 'precio_final': 0, 'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Ceuta', 'pais': 'Spain'},
 'usuario': {'ciudad': 'Ceuta',
             'fechaNacimiento': '1991-05-20',
             'pais': 'Spain',
             'sexo': 'H'}}
56
---------------------------------------------------------------
{'fecha': {'anyo': '2019', 'dia': '03', 'mes': '08'},
 'juego': {'clasificacion': 'AAA',
           'genero': ['adventure'],
           'precio': 5,
           'puntero_juego': 2596},
 'movimiento': {'cantidad': 1, 'precio_final': 0, 'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Ceuta', 'pais': 'Spain'},
 'usuario': {'ciudad': 'Ceuta',
             'fechaNacimiento': '1991-05-20',
             'pais': 'S

---------------------------------------------------------------
{'fecha': {'anyo': '2019', 'dia': '11', 'mes': '03'},
 'juego': {'clasificacion': 'INDIE',
           'genero': ['action', 'adventure'],
           'precio': 5,
           'puntero_juego': 2858},
 'movimiento': {'cantidad': 1, 'precio_final': 0, 'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Castelló', 'pais': 'Spain'},
 'usuario': {'ciudad': 'Castelló',
             'fechaNacimiento': '1997-07-15',
             'pais': 'Spain',
             'sexo': 'M'}}
88
---------------------------------------------------------------
{'fecha': {'anyo': '2019', 'dia': '13', 'mes': '05'},
 'juego': {'clasificacion': 'INDIE',
           'genero': ['strategy', 'simulation'],
           'precio': 30,
           'puntero_juego': 4950},
 'movimiento': {'cantidad': 1, 'precio_final': 0, 'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Pamplona', 'pais': 'Spain'},
 'usuario': {'ciudad': 'Pamplona',
             'fechaNacimiento': 

           'genero': ['action'],
           'precio': 12,
           'puntero_juego': 2853},
 'movimiento': {'cantidad': 1, 'precio_final': 0, 'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Santa Cruz de Tenerife', 'pais': 'Spain'},
 'usuario': {'ciudad': 'Santa Cruz de Tenerife',
             'fechaNacimiento': '1988-08-05',
             'pais': 'Spain',
             'sexo': 'M'}}
113
---------------------------------------------------------------
{'fecha': {'anyo': '2019', 'dia': '12', 'mes': '12'},
 'juego': {'clasificacion': 'AAA',
           'genero': ['adventure', 'strategy', 'early_access'],
           'precio': 10,
           'puntero_juego': 9516},
 'movimiento': {'cantidad': 1, 'precio_final': 0, 'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Santa Cruz de Tenerife', 'pais': 'Spain'},
 'usuario': {'ciudad': 'Santa Cruz de Tenerife',
             'fechaNacimiento': '1988-08-05',
             'pais': 'Spain',
             'sexo': 'M'}}
114
----------------------

...

---------------------------------------------------------------
{'fecha': {'anyo': '2019', 'dia': '08', 'mes': '02'},
 'juego': {'clasificacion': 'INDIE',
           'genero': ['action', 'casual', 'strategy', 'simulation'],
           'precio': 10,
           'puntero_juego': 6447},
 'movimiento': {'cantidad': 1, 'precio_final': 0, 'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Köflach', 'pais': 'Austria'},
 'usuario': {'ciudad': 'Köflach',
             'fechaNacimiento': '2000-08-20',
             'pais': 'Austria',
             'sexo': 'M'}}
1748
---------------------------------------------------------------
{'fecha': {'anyo': '2019', 'dia': '03', 'mes': '09'},
 'juego': {'clasificacion': 'AAA',
           'genero': [],
           'precio': 30,
           'puntero_juego': 1747},
 'movimiento': {'cantidad': 1, 'precio_final': 0, 'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Köflach', 'pais': 'Austria'},
 'usuario': {'ciudad': 'Köflach',
             'fechaNacimiento'

---------------------------------------------------------------
{'fecha': {'anyo': '2019', 'dia': '27', 'mes': '10'},
 'juego': {'clasificacion': 'INDIE',
           'genero': ['action'],
           'precio': 5,
           'puntero_juego': 941},
 'movimiento': {'cantidad': 1, 'precio_final': 0, 'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Brunn am Gebirge', 'pais': 'Austria'},
 'usuario': {'ciudad': 'Brunn am Gebirge',
             'fechaNacimiento': '2001-02-26',
             'pais': 'Austria',
             'sexo': 'M'}}
1777
---------------------------------------------------------------
{'fecha': {'anyo': '2019', 'dia': '07', 'mes': '11'},
 'juego': {'clasificacion': 'AAA',
           'genero': ['adventure', 'casual'],
           'precio': 3,
           'puntero_juego': 8953},
 'movimiento': {'cantidad': 1, 'precio_final': 0, 'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Brunn am Gebirge', 'pais': 'Austria'},
 'usuario': {'ciudad': 'Brunn am Gebirge',
            

---------------------------------------------------------------
{'dlc': {'precio': 30, 'tipo': ' trama adicional'},
 'fecha': {'anyo': '2019', 'dia': '13', 'mes': '10'},
 'movimiento': {'cantidad': 1, 'precio_final': 0, 'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Cartagena', 'pais': 'Spain'},
 'usuario': {'ciudad': 'Cartagena',
             'fechaNacimiento': '1992-12-13',
             'pais': 'Spain',
             'sexo': 'M'}}
1807
---------------------------------------------------------------
{'descuento': {'fecha_fin': '2018-12-29',
               'fecha_inicio': '2018-12-01',
               'porcentaje': 82},
 'dlc': {'precio': 25, 'tipo': ' cosmetico '},
 'fecha': {'anyo': '2019', 'dia': '16', 'mes': '09'},
 'movimiento': {'cantidad': 1, 'precio_final': 0, 'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Barcelona', 'pais': 'Spain'},
 'usuario': {'ciudad': 'Barcelona',
             'fechaNacimiento': '1992-02-23',
             'pais': 'Spain',
             'sexo

---------------------------------------------------------------
{'dlc': {'precio': 21, 'tipo': ' cosmetico '},
 'fecha': {'anyo': '2019', 'dia': '25', 'mes': '12'},
 'movimiento': {'cantidad': 1, 'precio_final': 0, 'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Saint-Dié-des-Vosges', 'pais': 'France'},
 'usuario': {'ciudad': 'Saint-Dié-des-Vosges',
             'fechaNacimiento': '1998-11-30',
             'pais': 'France',
             'sexo': 'H'}}
1834
---------------------------------------------------------------
{'descuento': {'fecha_fin': '2019-09-16',
               'fecha_inicio': '2019-08-19',
               'porcentaje': 93},
 'dlc': {'precio': 20, 'tipo': 'moneda virtual '},
 'fecha': {'anyo': '2019', 'dia': '08', 'mes': '08'},
 'movimiento': {'cantidad': 1, 'precio_final': 0, 'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Vichy', 'pais': 'France'},
 'usuario': {'ciudad': 'Vichy',
             'fechaNacimiento': '1988-01-08',
             'pais': 'France',
 

---------------------------------------------------------------
{'descuento': {'fecha_fin': '2017-02-19',
               'fecha_inicio': '2017-01-22',
               'porcentaje': 16},
 'dlc': {'precio': 12, 'tipo': ' cosmetico '},
 'fecha': {'anyo': '2019', 'dia': '08', 'mes': '11'},
 'movimiento': {'cantidad': 1, 'precio_final': 0, 'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Schwaz', 'pais': 'Austria'},
 'usuario': {'ciudad': 'Schwaz',
             'fechaNacimiento': '1988-10-20',
             'pais': 'Austria',
             'sexo': 'H'}}
1865
---------------------------------------------------------------
{'descuento': {'fecha_fin': '2018-12-29',
               'fecha_inicio': '2018-12-01',
               'porcentaje': 66},
 'dlc': {'precio': 13, 'tipo': ' cosmetico '},
 'fecha': {'anyo': '2019', 'dia': '17', 'mes': '09'},
 'movimiento': {'cantidad': 1, 'precio_final': 0, 'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Köflach', 'pais': 'Austria'},
 'usuario': {'ci